<a href="https://colab.research.google.com/github/AryanaBell/DS2002-ETL-Project/blob/main/DS2002_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import requests
import sqlite3

#Aryana Bellamkonda, Mariana Mendez, Caroline Nguyen
# 1. Data Ingestion

# Stock Data from Alpaca API
API_KEY = ""       # Your Alpaca API key
API_SECRET = ""  # Your Alpaca API secret key
symbol = "DAL"                          # Delta Airlines ticker
start_date = "2020-01-01"
end_date = "2020-12-31"

stock_url = f"https://data.alpaca.markets/v2/stocks/{symbol}/bars"
headers = {"APCA-API-KEY-ID": API_KEY, "APCA-API-SECRET-KEY": API_SECRET}
params = {"start": start_date, "end": end_date, "timeframe": "1Day"}
response = requests.get(stock_url, headers=headers, params=params)
response.raise_for_status()  # If the API call request fails for any reason, the code will stop

stock_df = pd.DataFrame(response.json().get("bars", []))
if not stock_df.empty:
    # Convert the timestamp column 't' to a proper date and store it in a new column "date"
    stock_df["date"] = pd.to_datetime(stock_df["t"]).dt.date
    # Remove the original timestamp column as it is no longer needed
    stock_df.drop(columns=["t"], inplace=True)
  # Print a summary (number of records and columns) of the stock data
print("Stock Data (Records, Columns):", stock_df.shape)

# COVID Data from local CSV file
try:
    covid_df = pd.read_csv("cumulative-deaths-and-cases-covid-19.csv") # Read the local CSV file containing COVID data
    covid_df.columns = covid_df.columns.str.strip()  # Clean column names
    # Filter for US data using the "Entity" column and force a copy to avoid SettingWithCopyWarning
    covid_df = covid_df[covid_df["Entity"] == "United States"].copy()
    # Rename the longer column names to make it easier
    covid_df.rename(columns={"Total confirmed cases of COVID-19": "total_cases",
                             "Total confirmed deaths due to COVID-19": "total_deaths"}, inplace=True)
    # Convert the "Day" column to a proper date and add as "date"
    covid_df["date"] = pd.to_datetime(covid_df["Day"]).dt.date
    print("COVID Data (Records, Columns):", covid_df.shape)
except Exception as e:
    print("Error reading local COVID data:", e) #Print error message for issues reading the csv file
    covid_df = pd.DataFrame()

# 2. Data Transformation and Merging

# Merge on the common "date" column and drop the redundant "Day" column
merged_df = pd.merge(stock_df, covid_df, on="date", how="inner")
merged_df = merged_df.drop(columns=["Day"])
# Print the Records and Columns and list the column names of the merged dataset
print("Merged Data (Records, Columns):", merged_df.shape)
print("Columns of Merged Dataset:", merged_df.columns.tolist())

# 3. Data Analysis and Summarization
# Calculate the correlation between the stock's closing price ("c") and COVID total cases ("total_cases")

if "c" in merged_df.columns and "total_cases" in merged_df.columns:
    corr = merged_df["c"].corr(merged_df["total_cases"])
    print("Correlation between closing price and total confirmed cases:", corr)
else:
    print("Required columns for correlation analysis are missing.") #Error message

print("\nSummary Statistics of Merged Data:")
print(merged_df.describe()) #Prints statistics for the merged data set

# 4. Data Storage: Output Format Selection
# Asks the user for the desired output format: csv, json, or sql (default is sql)
format = input("\nEnter desired output format (csv, json, sql) [default: sql]: ").strip().lower()
if format not in ["csv", "json", "sql"]:
    format = "sql"
if format == "csv":
    merged_df.to_csv("merged_data.csv", index=False)
    print("Data saved as merged_data.csv")
elif format == "json":
    merged_df.to_json("merged_data.json", orient="records", date_format="iso")
    print("Data saved as merged_data.json")
else:
    conn = sqlite3.connect("merged_data.db")
    merged_df.to_sql("merged_data", conn, if_exists="replace", index=False)
    conn.close()
    print("Data saved in merged_data.db as a SQLite table")
    print(merged_df) #Prints the sql database


Stock Data (Records, Columns): (253, 8)
COVID Data (Records, Columns): (1885, 6)
Merged Data (Records, Columns): (251, 12)
Columns of Merged Dataset: ['c', 'h', 'l', 'n', 'o', 'v', 'vw', 'date', 'Entity', 'Code', 'total_deaths', 'total_cases']
Correlation between closing price and total confirmed cases: -0.02934458777668341

Summary Statistics of Merged Data:
                c           h           l              n           o  \
count  251.000000  251.000000  251.000000     251.000000  251.000000   
mean    34.788725   35.672010   33.952502  169991.394422   34.929103   
std     11.401663   11.348368   11.419814  118045.096721   11.389011   
min     19.190000   19.540000   17.510000   27689.000000   18.800000   
25%     26.900000   27.610000   26.070000   90997.000000   26.700000   
50%     31.140000   31.960000   30.480000  132470.000000   31.250000   
75%     40.405000   41.040000   39.895000  214935.500000   40.500000   
max     62.030000   62.480000   61.865000  742962.000000   62.